In [ ]:
import spatialmeta as smt
import numpy as np
import pandas as pd
import scanpy as sc
from pyimzml.ImzMLParser import ImzMLParser

In [ ]:
SM_file = '/media/duan/sda2/MALDI/Data/LPS_01/2.imzML/Aligned_imzML/1.imzML'
p = ImzMLParser(SM_file)

In [ ]:
mz_reference = smt.pp.get_mz_reference(p,ppm_tolerance=10)
mz_reference

In [ ]:
adata_SM_pos = smt.pp.read_sm_imzml_as_anndata(p,mz_reference)

In [ ]:
adata_SM_pos

In [ ]:
adata_SM_pos.obs

In [ ]:
adata_SM_pos.var['m/z'] = [float(x) for x in adata_SM_pos.var['m/z']]

In [ ]:
adata_SM_pos.obs['x_coord_original'].mean()

In [ ]:
adata_SM_pos.obs['Saline'] = adata_SM_pos.obs['x_coord_original'] > 800

In [ ]:
sc.pl.embedding(adata_SM_pos[adata_SM_pos.obs['Saline']], basis = 'spatial', color = 'mean_intensity')

In [ ]:
sc.pl.embedding(adata_SM_pos[~adata_SM_pos.obs['Saline']], basis = 'spatial', color = 'mean_intensity')

In [ ]:
adata_LPS = adata_SM_pos[~adata_SM_pos.obs['Saline']].copy()

In [ ]:
adata_LPS.obsm['spatial']

In [ ]:
adata_LPS.obs['x_scaled'] = adata_LPS.obs['x_coord_original'] - 1
adata_LPS.obs['y_scaled'] = adata_LPS.obs['y_coord_original'] - 1

In [ ]:
adata_LPS.obs['x_scaled'] = adata_LPS.obs['x_scaled'] * 61.5384
adata_LPS.obs['y_scaled'] = adata_LPS.obs['y_scaled'] * 61.5384

In [ ]:
adata_LPS.obs

In [ ]:
adata_LPS.obs['x_scaled'].max()

In [ ]:
adata_LPS.obs['y_scaled'].max()

In [ ]:
adata_LPS.obs['x_scaled'] = adata_LPS.obs['x_scaled'] + 40
adata_LPS.obs['y_scaled'] = adata_LPS.obs['y_scaled'] + 40

In [ ]:
adata_LPS.write_h5ad("LPS_Scaled_raw.h5ad")

In [ ]:
from shapely import Polygon, affinity
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import tifffile

In [ ]:
DIC_img = tifffile.imread("/media/duan/sda2/MALDI/Data/20250727_LPS_MALDI/Stitched_DIC.tif")
DIC_img.shape

In [ ]:
img = np.zeros_like(DIC_img)

In [ ]:
for idx, row in adata.obs.iterrows():
    
    TopLeft_x = round(row['y_aligned'])
    TopLeft_y = round(row['x_aligned'])
    
    if TopLeft_x > 0 and TopLeft_x + 40 < 24187 and TopLeft_y >0 and TopLeft_y + 40 < 44316:
    
        img[TopLeft_x: TopLeft_x + 40, TopLeft_y: TopLeft_y + 40] = row['scaled_total_intensity']

In [ ]:
tifffile.imwrite("LPS_total_intensity_aligned.tif", img.astype(np.uint8))

In [ ]:
import cv2

In [ ]:
M = np.array([[1.0007454663706765, 0.008320817632502187, -31.366865549945945], [-0.008336573756612373, 0.9988540574740241, -3278.7280880942467]])

In [ ]:
new_points = adata.obs.loc[:, ['y_scaled', 'x_scaled']].to_numpy()

In [ ]:
transformed_new_points = cv2.transform(new_points.reshape(-1, 1, 2), M).reshape(-1, 2)

In [ ]:
adata.obs['x_aligned'] = transformed_new_points[:, 1]
adata.obs['y_aligned'] = transformed_new_points[:, 0]

In [ ]:
sum(adata.obs['x_aligned'] < 0)

In [ ]:
DIC_img = tifffile.imread("/media/duan/sda2/MALDI/Data/20250727_LPS_MALDI/Stitched_DIC.tif")
DIC_img.shape

In [ ]:
img = np.zeros_like(DIC_img)

In [ ]:
adata

In [ ]:
for idx, row in adata.obs.iterrows():
    
    TopLeft_x = round(row['y_aligned'])
    TopLeft_y = round(row['x_aligned'])
    
    if TopLeft_x > 0 and TopLeft_x + 40 < 24187 and TopLeft_y >0 and TopLeft_y + 40 < 44316:
    
        img[TopLeft_x: TopLeft_x + 40, TopLeft_y: TopLeft_y + 40] = row['scaled_intensity']

In [ ]:
tifffile.imwrite("LPS_scaled_intensity_aligned.tif", img.astype(np.uint8))

In [ ]:
adata.write_h5ad("LPS_Scaled_Reg1.h5ad")

In [ ]:
from read_roi import read_roi_file
from shapely import Polygon, affinity
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import tifffile
import pandas as pd
import cv2

import scanpy as sc
from roifile import ImagejRoi, ROI_TYPE, ROI_OPTIONS, roiwrite

In [ ]:
adata = sc.read_h5ad("LPS_Scaled_Reg1.h5ad")
adata.obs

In [ ]:
M = np.array([[0.995782858272422, -0.0018972998360801893, 160.21749144068235], [0.0018914893645645132, 0.9988418064442617, 7.464203150175308]])

In [ ]:
new_points = adata.obs.loc[:, ['y_aligned', 'x_aligned']].to_numpy()

In [ ]:
transformed_new_points = cv2.transform(new_points.reshape(-1, 1, 2), M).reshape(-1, 2)

In [ ]:
adata.obs['x_aligned'] = transformed_new_points[:, 1]
adata.obs['y_aligned'] = transformed_new_points[:, 0]

In [ ]:
adata.obs

In [ ]:
from spatialdata import read_zarr
import sopa.segmentation as sseg
import geopandas as gpd
import scanpy as sc
from shapely import Polygon, Point
import numpy as np
from spatialdata.models import Image2DModel, PointsModel,ShapesModel,TableModel

In [ ]:
adata = sc.read_h5ad("/media/duan/sda2/MALDI/Data/LPS_01/LPS_Scaled_Reg2.h5ad")
adata

In [ ]:
sdata = read_zarr("/media/duan/sda2/MALDI/Data/20250727_LPS_MALDI/Result/raw_sdata.zarr")
sdata

In [ ]:
polygon_list = []

for idx, row in adata.obs.iterrows():
    
    TopLeft_x = row['x_aligned']
    TopLeft_y = row['y_aligned']
    
    tmp_polygon = Polygon(np.array([[TopLeft_x, TopLeft_y], [TopLeft_x + 40, TopLeft_y], [TopLeft_x + 40, TopLeft_y + 40], [TopLeft_x, TopLeft_y + 40], [TopLeft_x, TopLeft_y]]))

    polygon_list.append(tmp_polygon)

In [ ]:
Ablation_Marks = gpd.GeoDataFrame({"geometry":polygon_list})
Ablation_Marks

In [ ]:
sdata.shapes['Ablation_Mark'] = ShapesModel.parse(Ablation_Marks)
sdata

In [ ]:
adata = sseg.aggregation.count_transcripts(sdata, gene_column = "gene", shapes_key='Ablation_Mark', points_key='dist_coord')

In [ ]:
adata.obs["x"] = sdata['Ablation_Mark'].apply(lambda x: x.centroid.x).to_numpy()
adata.obs["y"] = sdata['Ablation_Mark'].apply(lambda x: x.centroid.y).to_numpy()
adata.obs["area"] = sdata['Ablation_Mark'].apply(lambda x: x.area).to_numpy()

In [ ]:
adata.write_h5ad("Result/FISH_in_Ablation_raw.h5ad")

In [ ]:
mdata = sc.read_h5ad("/media/duan/sda2/MALDI/Data/LPS_01/LPS_Scaled_Reg2.h5ad")
mdata

In [ ]:
mdata.obs

In [ ]:
adata.obs

In [ ]:
from scipy.spatial import KDTree

In [ ]:
tree = KDTree(mdata.obs[['x_aligned', 'y_aligned']])

In [ ]:
sm_index = []
for _,row in adata.obs.iterrows():
    _, idx = tree.query([[row['x'],row['y']]])
    sm_index.append(str(idx[0]))

In [ ]:
adata.obs['index_sm'] = sm_index

In [ ]:
adata.obs

In [ ]:
adata.write_h5ad("Result/FISH_in_Ablation_raw.h5ad")

In [ ]:
import scanpy as sc
from celllens.utils import *
import os
from tqdm import tqdm
from celllens.preprocessing import *
from celllens.datasets import *
from celllens.celllens import *
import pandas as pd

In [ ]:
adata = sc.read_h5ad("Result/FISH_in_Ablation_raw.h5ad")
adata

In [ ]:
NP_Pad = pd.read_csv("/media/duan/sda2/MALDI/Data/20250727_Saline_MALDI/NP_Fold_V58.csv", index_col=0)

In [ ]:
adata.var_names = [x[0:-2] if x.endswith("_s") else x for x in adata.var_names]

In [ ]:
tmpmatrix = adata.to_df().copy()

In [ ]:
tmpmatrix = tmpmatrix.astype('float32')

In [ ]:
import numpy as np

for col in tmpmatrix.columns:
    # tmpmatrix.loc[:,col] = tmpmatrix.loc[:,col] * np.log2(NP_Pad.loc[col, 'Fold'] + 1)
    tmpmatrix.loc[:,col] = tmpmatrix.loc[:,col] * NP_Pad.loc[col, 'Fold']

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
adata.layers['counts'] = adata.X.copy()
adata.X = csr_matrix(tmpmatrix)
adata.layers['counts_corrected'] = adata.X.copy()

In [ ]:
adata.obsm['spatial'] = adata.obs[['x', 'y']].to_numpy()
adata.obsm["spatial"][:,1] =  0 - adata.obsm["spatial"][:,1]

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, inplace=True, log1p=True, percent_top=None
)

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts"],
    jitter=0.4,
    multi_panel=True,
)

In [ ]:
adata.obs['total_counts'].mean()

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
np.percentile(adata.obs['total_counts'], 10)

In [ ]:
sc.pp.filter_cells(adata, min_genes=3)
sc.pp.filter_cells(adata, max_counts=250)
sc.pp.filter_cells(adata, min_counts=10)
adata

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
# Normalizing to library size
sc.pp.normalize_total(adata, target_sum=110)
# Logarithmize the data
sc.pp.log1p(adata)

In [ ]:
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

In [ ]:
# Using the igraph implementation and a fixed number of iterations can be significantly faster, especially for larger datasets
sc.tl.leiden(adata, flavor="igraph",n_iterations=2)

In [ ]:
adata.obsm['spatial'].min(axis = 0) - adata.obsm['spatial'].max(axis = 0)

In [ ]:
df = adata.to_df(layer = 'counts_corrected')
df

In [ ]:
df['x'] = adata.obs['x']
df['y'] = adata.obs['y']
#df['area'] = adata.obs['area']
df['leiden'] = adata.obs['leiden']

In [ ]:
df

In [ ]:
Rep2_dataset = LENS_Dataset(df,
                              features_list=adata.var_names.to_list(),
                              nbhd_composition=15,
                              feature_neighbor=15,
                              spatial_neighbor=15,
                              path2img=None,
                             use_transform = False) # Not used -  processed image saving location - in later steps

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Rep2_dataset.initialize(cent_x="x", # x loc of cells
                          cent_y="y", # y loc of cells
                          celltype="feature_labels", # default option - run leiden to initialize the clusters
                          pca_components=25, # PCA components to use, user decide 
                          cluster_res=2.0) # leiden initialization resolution

In [ ]:
# check the training label
Rep2_dataset.dual_labels.shape # should be cluster number x 2

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
Rep2_celllens = CellLENS(Rep2_dataset,
                           device,
                           cnn_model=False, ### NOTE here set to no cnn model
                           cnn_latent_dim=128,
                           gnn_latent_dim=32,
                          ) # generally these parameters no need to change

In [ ]:
Rep2_celllens.get_lens_embedding(round=5,
                                   k=32,
                                   learning_rate=1e-3,
                                   n_epochs=5000,
                                   loss_fn='MSELoss',
                                   OptimizerAlg='Adam',
                                   optimizer_kwargs={},
                                   SchedulerAlg=None,
                                   scheduler_kwargs={},
                                   verbose=True) # generally these parameters do not need to change

In [ ]:
# the resulting CellLENS embedding is stored here:
pd.DataFrame(Rep2_celllens.lens_embedding)

In [ ]:
Rep2_celllens.get_lens_clustering(neighbor=15, # standard leiden parameter
                                    resolution=1.0, # resolution for leiden - specific
                                    entropy_threshold=0.75, # CellLENS parameter - generally no change needed
                                    concen_threshold=1, # CellLENS parameter - generally no change needed
                                    max_breaks=3, # CellLENS parameter - generally no change needed
                                    size_lim=50 # CellLENS parameter - smallest cluster size allowed
                                   )

In [ ]:
adata.obsm['CellLENS'] = Rep2_celllens.lens_embedding
adata.obs['lens_cluster'] = Rep2_celllens.lens_clustering

In [ ]:
adata.write_h5ad("/media/duan/sda2/MALDI/Data/20250727_LPS_MALDI/Result/FISH_in_Ablation_CellLens.h5ad")

In [ ]:
adata = sc.read_h5ad("/media/duan/sda2/MALDI/Data/LPS_01/LPS_Scaled_Reg2.h5ad")
adata

In [ ]:
bdf = pd.read_csv("/media/duan/sda2/MALDI/Data/LPS_01/BACKGROUND_mz.csv", header=None)
bdf

In [ ]:
mz_BLACK_LIST = bdf[0].to_numpy(dtype = str)

In [ ]:
adata = adata[:, [x for x in adata.var_names if x not in mz_BLACK_LIST]].copy()

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
adata

In [ ]:
adata.layers['RMS'] = adata.X.copy()

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, inplace=True, log1p=True, percent_top=None
)

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts"],
    jitter=0.4,
    multi_panel=True,
)

In [ ]:
adata.obs['total_counts'].mean()

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
adata.obs['total_counts'].median()

In [ ]:
np.percentile(adata.obs['total_counts'], 5)

In [ ]:
sc.pp.filter_cells(adata, min_genes=400)
sc.pp.filter_cells(adata, max_counts=60000)
sc.pp.filter_cells(adata, min_counts=8000)
adata

In [ ]:
# Logarithmize the data
sc.pp.log1p(adata)

In [ ]:
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

In [ ]:
# Using the igraph implementation and a fixed number of iterations can be significantly faster, especially for larger datasets
sc.tl.leiden(adata, flavor="igraph",n_iterations=2)

In [ ]:
adata.obsm['spatial'].min(axis = 0) - adata.obsm['spatial'].max(axis = 0)

In [ ]:
df = adata.to_df(layer = 'RMS')
df

In [ ]:
df['x'] = adata.obs['x_aligned']
df['y'] = adata.obs['y_aligned']
#df['area'] = adata.obs['area']
df['leiden'] = adata.obs['leiden']

In [ ]:
df

In [ ]:
Rep2_dataset = LENS_Dataset(df,
                              features_list=adata.var_names.to_list(),
                              nbhd_composition=15,
                              feature_neighbor=15,
                              spatial_neighbor=15,
                              path2img=None,
                             use_transform = False) # Not used -  processed image saving location - in later steps

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Rep2_dataset.initialize(cent_x="x", # x loc of cells
                          cent_y="y", # y loc of cells
                          celltype="feature_labels", # default option - run leiden to initialize the clusters
                          pca_components=25, # PCA components to use, user decide 
                          cluster_res=2.0) # leiden initialization resolution

In [ ]:
# check the training label
Rep2_dataset.dual_labels.shape # should be cluster number x 2

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
Rep2_celllens = CellLENS(Rep2_dataset,
                           device,
                           cnn_model=False, ### NOTE here set to no cnn model
                           cnn_latent_dim=128,
                           gnn_latent_dim=32,
                          ) # generally these parameters no need to change

In [ ]:
Rep2_celllens.get_lens_embedding(round=5,
                                   k=32,
                                   learning_rate=1e-3,
                                   n_epochs=5000,
                                   loss_fn='MSELoss',
                                   OptimizerAlg='Adam',
                                   optimizer_kwargs={},
                                   SchedulerAlg=None,
                                   scheduler_kwargs={},
                                   verbose=True) # generally these parameters do not need to change

In [ ]:
# the resulting CellLENS embedding is stored here:
pd.DataFrame(Rep2_celllens.lens_embedding)

In [ ]:
Rep2_celllens.get_lens_clustering(neighbor=15, # standard leiden parameter
                                    resolution=1.0, # resolution for leiden - specific
                                    entropy_threshold=0.75, # CellLENS parameter - generally no change needed
                                    concen_threshold=1, # CellLENS parameter - generally no change needed
                                    max_breaks=3, # CellLENS parameter - generally no change needed
                                    size_lim=50 # CellLENS parameter - smallest cluster size allowed
                                   )

In [ ]:
adata.obsm['CellLENS'] = Rep2_celllens.lens_embedding
adata.obs['lens_cluster'] = Rep2_celllens.lens_clustering

In [ ]:
adata.write_h5ad("/media/duan/sda2/MALDI/Data/20250727_LPS_MALDI/Result/MALDI_in_Ablation_CellLens.h5ad")

In [ ]:
adata_st = sc.read_h5ad("/media/duan/sda2/MALDI/Data/20250727_LPS_MALDI/Result/FISH_in_Ablation_CellLens.h5ad")

In [ ]:
adata_sm = sc.read_h5ad("/media/duan/sda2/MALDI/Data/20250727_LPS_MALDI/Result/MALDI_in_Ablation_CellLens.h5ad")

In [ ]:
sc.pp.highly_variable_genes(adata_sm, n_top_genes=110)

In [ ]:
adata_sm

In [ ]:
adata_sm = adata_sm[:, adata_sm.var['highly_variable']].copy()

In [ ]:
adata_sm

In [ ]:
adata_st.X = adata_st.layers['counts_corrected'].copy()

In [ ]:
sc.pp.normalize_total(adata_st, target_sum=110)

In [ ]:
adata_sm.X = adata_sm.layers['RMS'].copy()

In [ ]:
sc.pp.normalize_total(adata_sm, target_sum=110)

In [ ]:
df1 = adata_st.to_df()

In [ ]:
df2 = adata_sm.to_df()

In [ ]:
cidx = [idx for idx in df1.index if idx in df2.index]
len(cidx)

In [ ]:
df1 = df1.loc[cidx,:].copy()
df2 = df2.loc[cidx,:].copy()

In [ ]:
df1

In [ ]:
df2

In [ ]:
new_df = pd.concat([df1, df2], axis = 1)

In [ ]:
adata = sc.AnnData(new_df)
adata

In [ ]:
st_raw = sc.read_h5ad("Result/FISH_in_Ablation_raw.h5ad")
st_raw

In [ ]:
adata.obs['x'] = st_raw.obs.loc[adata.obs_names,'x']
adata.obs['y'] = st_raw.obs.loc[adata.obs_names,'y']

In [ ]:
adata.obsm['spatial'] = adata.obs[['x', 'y']].to_numpy()
adata.obsm["spatial"][:,1] =  0 - adata.obsm["spatial"][:,1]

In [ ]:
# Logarithmize the data
sc.pp.log1p(adata)

In [ ]:
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

In [ ]:
# Using the igraph implementation and a fixed number of iterations can be significantly faster, especially for larger datasets
sc.tl.leiden(adata, flavor="igraph",n_iterations=2)

In [ ]:
df = adata.to_df()
df

In [ ]:
df['x'] = adata.obs['x']
df['y'] = adata.obs['y']
# df['area'] = adata.obs['area']
# df['leiden'] = adata.obs['leiden']

In [ ]:
df

In [ ]:
Rep2_dataset = LENS_Dataset(df,
                              features_list=adata.var_names.to_list(),
                              nbhd_composition=15,
                              feature_neighbor=15,
                              spatial_neighbor=15,
                              path2img=None,
                             use_transform = False) # Not used -  processed image saving location - in later steps

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Rep2_dataset.initialize(cent_x="x", # x loc of cells
                          cent_y="y", # y loc of cells
                          celltype="feature_labels", # default option - run leiden to initialize the clusters
                          pca_components=25, # PCA components to use, user decide 
                          cluster_res=2.0) # leiden initialization resolution

In [ ]:
# check the training label
Rep2_dataset.dual_labels.shape # should be cluster number x 2

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
Rep2_celllens = CellLENS(Rep2_dataset,
                           device,
                           cnn_model=False, ### NOTE here set to no cnn model
                           cnn_latent_dim=128,
                           gnn_latent_dim=32,
                          ) # generally these parameters no need to change

In [ ]:
Rep2_celllens.get_lens_embedding(round=5,
                                   k=32,
                                   learning_rate=1e-3,
                                   n_epochs=5000,
                                   loss_fn='MSELoss',
                                   OptimizerAlg='Adam',
                                   optimizer_kwargs={},
                                   SchedulerAlg=None,
                                   scheduler_kwargs={},
                                   verbose=True) # generally these parameters do not need to change

In [ ]:
# the resulting CellLENS embedding is stored here:
pd.DataFrame(Rep2_celllens.lens_embedding)

In [ ]:
adata.obsm['CellLENS'] = Rep2_celllens.lens_embedding
adata.obs['lens_cluster'] = Rep2_celllens.lens_clustering

In [ ]:
adata.write_h5ad("/media/duan/sda2/MALDI/Data/20250727_LPS_MALDI/Result/MALDI_FISH_in_Ablation_CellLens.h5ad")

In [ ]:
import MENDER
import scanpy as sc
import pandas as pd
import numpy as np
from sklearn.metrics import *

In [ ]:
adata = sc.read_h5ad('Result/MALDI_in_Ablation_CellLens.h5ad')
adata

In [ ]:
adata.obsm['spatial'] = adata.obs[['x', 'y']].to_numpy()
adata.obsm["spatial"][:,1] =  0 - adata.obsm["spatial"][:,1]

In [ ]:
# Normalizing to library size
sc.pp.normalize_total(adata, target_sum=110)
# Logarithmize the data
sc.pp.log1p(adata)

In [ ]:
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

In [ ]:
# Using the igraph implementation and a fixed number of iterations can be significantly faster, especially for larger datasets
sc.tl.leiden(adata, resolution=2.0)

In [ ]:
adata = sc.read_h5ad("/media/duan/sda2/MALDI/Data/20250727_LPS_MALDI/Result/MALDI_FISH_in_Ablation_CellLens.h5ad")
adata

In [ ]:
scale = 6
radius = 90

# main body of MENDER
msm = MENDER.MENDER_single(
    adata,
    # determine which cell state to use
    # we use the cell state got by Leiden
    ct_obs='lens_cluster',
    random_seed=666
)


# set the MENDER parameters

msm.set_MENDER_para(
    # default of n_scales is 6
    n_scales=scale,

    # for single cell data, nn_mode is set to 'radius'
    nn_mode='radius',

    # default of n_scales is 15 um (see the manuscript for why).
    # MENDER also provide a function 'estimate_radius' for estimating the radius
    nn_para=radius,

)
# construct the context representation
msm.run_representation(
    200
    # the number of processings
)

# set the spatial clustering parameter
# positive values for the expected number of domains
# negative values for the clustering resolution
msm.run_clustering_normal(-1.0)

In [ ]:
msm.run_clustering_normal(-0.3)

In [ ]:
adata.obs["MENDER"] = msm.adata_MENDER.obs["MENDER"]

In [ ]:
del adata.uns['MENDER_colors']

In [ ]:
sc.pl.embedding(adata, basis="spatial", color=["MENDER"], size = 10)

In [ ]:
adata.write_h5ad("Result/MENDER_MALDI_FISH_LPS.h5ad")

In [ ]:
adata = sc.read_h5ad('Result/MALDI_in_Ablation_CellLens.h5ad')
adata

In [ ]:
adata.obsm['spatial'] = adata.obs[['x_aligned', 'y_aligned']].to_numpy()
adata.obsm["spatial"][:,1] =  0 - adata.obsm["spatial"][:,1]

In [ ]:
adata.obsm['spatial']

In [ ]:
scale = 6
radius = 90

# main body of MENDER
msm = MENDER.MENDER_single(
    adata,
    # determine which cell state to use
    # we use the cell state got by Leiden
    ct_obs='lens_cluster',
    random_seed=666
)


# set the MENDER parameters

msm.set_MENDER_para(
    # default of n_scales is 6
    n_scales=scale,

    # for single cell data, nn_mode is set to 'radius'
    nn_mode='radius',

    # default of n_scales is 15 um (see the manuscript for why).
    # MENDER also provide a function 'estimate_radius' for estimating the radius
    nn_para=radius,

)
# construct the context representation
msm.run_representation(
    200
    # the number of processings
)

# set the spatial clustering parameter
# positive values for the expected number of domains
# negative values for the clustering resolution
msm.run_clustering_normal(-1.0)

In [ ]:
msm.run_clustering_normal(-0.3)

In [ ]:
adata.obs["MENDER"] = msm.adata_MENDER.obs["MENDER"]

In [ ]:
sc.set_figure_params(figsize = (11,5.9))
sc.pl.embedding(adata, basis="spatial", color=["MENDER"], size = 5)

In [ ]:
adata.write_h5ad("Result/MENDER_MALDI_LPS.h5ad")

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np

In [ ]:
adata_sm = sc.read_h5ad("Result/MENDER_MALDI_LPS.h5ad")
adata_st = sc.read_h5ad("Result/MENDER_FISH_LPS.h5ad")
adata = sc.read_h5ad("Result/MENDER_MALDI_FISH_LPS.h5ad")

In [ ]:
adata.obs['MENDER'].cat.categories

In [ ]:
adata.uns['MENDER_colors']

In [ ]:
sc.set_figure_params(figsize = (11,6), scanpy = False, dpi_save=300)
sc.pl.embedding(adata, basis="spatial", color=["MENDER"], size = 5, save = 'MENDER_FISH_MALDI.pdf', frameon = False, legend_loc=None, title = 'OpenFISH + MALDI')

In [ ]:
MALDI_FISH_color = {x:y for x,y in zip(adata.obs['MENDER'].cat.categories, adata.uns['MENDER_colors'])}

In [ ]:
FISH_color = {'0': MALDI_FISH_color['1'], '1': MALDI_FISH_color['7'], '2': MALDI_FISH_color['0'], '3': MALDI_FISH_color['9'], '4': MALDI_FISH_color['6'],
             '5': MALDI_FISH_color['15'], '6': MALDI_FISH_color['28'], '7': MALDI_FISH_color['4'], '8': MALDI_FISH_color['23'], '9': MALDI_FISH_color['18'],
             '10': MALDI_FISH_color['11'], '11': MALDI_FISH_color['14'], '12': MALDI_FISH_color['22'], '13': MALDI_FISH_color['3'], '14': MALDI_FISH_color['24'],
             '15': MALDI_FISH_color['17'], '16': MALDI_FISH_color['35'], '17': MALDI_FISH_color['33'], '18': MALDI_FISH_color['32'], '19': MALDI_FISH_color['27'],
             '20': MALDI_FISH_color['40']}

In [ ]:
adata_st.uns['MENDER_colors']

In [ ]:
FISH_color = {x:y for x,y in zip(adata_st.obs['MENDER'].cat.categories, adata_st.uns['MENDER_colors'])}

In [ ]:
FISH_color = {'0': '#1ce6ff',
 '1': '#ffdbe5',
 '2': '#ffff00',
 '3': '#0000a6',
 '4': '#a30059',
 '5': '#5a0007',
 '6': '#ff90c9',
 '7': '#008941',
 '8': '#61615a',
 '9': '#1b4400',
 '10': '#b79762',
 '11': '#997d87',
 '12': '#ff2f80',
 '13': '#ff4a46',
 '14': '#ba0900',
 '15': '#6a3a4c',
 '16': '#300018',
 '17': '#7b4f4b',
 '18': '#000035',
 '19': '#ffaa92',
 '20': '#ffb500', '21': '#4a3b53',
 '22': '#ff2f80',
 '23': '#61615a',
 '24': '#ba0900',
 '25': '#6b7900',
 '26': '#00c2a0',
 '27': '#ffaa92',
 '28': '#ff90c9',
 '29': '#b903aa',
 '30': '#d16100',
 '31': '#ddefff',
 '32': '#000035',
 '33': '#7b4f4b',
 '34': '#a1c299',
 '35': '#300018',
 '36': '#0aa6d8',
 '37': '#013349',
 '38': '#00846f',
 '39': '#372101',
 '40': '#ffb500',
 '41': '#c2ffed',
 '42': '#a079bf',
 '43': '#cc0744',
 '44': '#c0b9b2',
 '45': '#c2ff99'}

In [ ]:
sc.set_figure_params(figsize = (11,6), scanpy = False, dpi_save=300)
sc.pl.embedding(adata_st, basis="spatial", color=["MENDER"], size = 8, save = 'MENDER_FISH.pdf', frameon = False, legend_loc=None, title = 'OpenFISH', palette=FISH_color)

In [ ]:
MALDI_color = {'0': '#ffff00',
 '1': '#b79762',
 '2': '#0000a6',
 '3': '#a30059',
 '4': '#63ffac',
 '5': '#ff4a46',
 '6': '#ff34ff',
 '7': '#ffdbe5',
 '8': '#008941',
 '9': '#6a3a4c',
 '10': '#7a4900',
 '11': '#ff90c9',
 '12': '#6b7900',
 '13': '#b79762',
 '14': '#ff34ff',
 '15': '#006fa6',
 '16': '#997d87',
 '17': '#1ce6ff',
 '18': '#8fb0ff',
 '19': '#004d43',
 '20': '#ba0900',
 '21': '#ff4a46',
 '22': '#ffdbe5',
 '23': '#8fb0ff',
 '24': '#b79762',
 '25': '#000035','26': '#00c2a0',
 '27': '#ffaa92',
 '28': '#ff90c9',
 '29': '#b903aa',
 '30': '#d16100',
 '31': '#ddefff',
 '32': '#000035',
 '33': '#7b4f4b',
 '34': '#a1c299',
 '35': '#300018',
 '36': '#0aa6d8',
 '37': '#013349',
 '38': '#00846f',
 '39': '#372101',
 '40': '#ffb500',
 '41': '#c2ffed',
 '42': '#a079bf',
 '43': '#cc0744',
 '44': '#c0b9b2',
 '45': '#c2ff99',
 '46': '#001e09',
 '47': '#00489c',
 '48': '#6f0062',
 '49': '#0cbd66',
 '50': '#eec3ff',
 '51': '#456d75',
 '52': '#b77b68',
 '53': '#7a87a1',
 '54': '#788d66',
 '55': '#885578',
 '56': '#fad09f',
 '57': '#ff8a9a',
 '58': '#d157a0',
 '59': '#bec459',
 '60': '#456648',
 '61': '#0086ed',
 '62': '#886f4c',
 '63': '#34362d',
 '64': '#b4a8bd',
 '65': '#00a6aa',
 '66': '#452c2c',
 '67': '#636375',
 '68': '#a3c8c9',
 '69': '#ff913f',
 '70': '#938a81',
 '71': '#575329',
 '72': '#00fecf',
 '73': '#b05b6f',
 '74': '#8cd0ff',
 '75': '#3b9700',
 '76': '#04f757',
 '77': '#c8a1a1',
 '78': '#1e6e00',
 '79': '#7900d7',
 '80': '#a77500',
 '81': '#6367a9',
 '82': '#a05837',
 '83': '#6b002c',
 '84': '#772600',
 '85': '#d790ff',
 '86': '#9b9700',
 '87': '#549e79',
 '88': '#fff69f',
 '89': '#201625',
 '90': '#72418f',
 '91': '#bc23ff',
 '92': '#99adc0',
 '93': '#3a2465',
 '94': '#922329',
 '95': '#5b4534',
 '96': '#fde8dc'}

In [ ]:
sc.set_figure_params(figsize = (11,6), scanpy = False, dpi_save=300)
sc.pl.embedding(adata_sm, basis="spatial", color=["MENDER"], size = 3, palette=MALDI_color, save = 'MENDER_MALDI.pdf', frameon = False, legend_loc=None, title = 'MALDI')

In [ ]:
cortex_polygon = pd.read_csv("Cortex_Shapes.csv")
cortex_polygon

In [ ]:
from shapely import Polygon, Point
import numpy as np

In [ ]:
polygon = Polygon((cortex_polygon[['axis-1', 'axis-0']]).to_numpy())
polygon

In [ ]:
roi = ImagejRoi.frompoints(np.array(cortex_polygon[['axis-1', 'axis-0']]))
roi.roitype = ROI_TYPE.POLYGON
roi.name = 'Cortex'
roi.options |= ROI_OPTIONS.SHOW_LABELS

In [ ]:
roiwrite('LPS_MALDI_Cortex_forFiji.roi', roi, mode='w')

In [ ]:
adata.obs['Cortex'] = adata.obs[['x_aligned', 'y_aligned']].apply(lambda row: polygon.contains(Point([row['x_aligned'],row['y_aligned']])), axis = 1)

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'Cortex', size = 10)

In [ ]:
tmp = adata[adata.obs['Cortex']].copy()

In [ ]:
tmp.obs

In [ ]:
rois = []

for idx, row in tmp.obs.iterrows():
    
    TopLeft_x = row['x_aligned']
    TopLeft_y = row['y_aligned']
    
    arr = np.array([[TopLeft_x, TopLeft_y], [TopLeft_x + 40, TopLeft_y], [TopLeft_x + 40, TopLeft_y + 40], [TopLeft_x, TopLeft_y + 40], [TopLeft_x, TopLeft_y]])

    roi = ImagejRoi.frompoints(arr)
    roi.roitype = ROI_TYPE.POLYGON
    roi.name = idx
    roi.options |= ROI_OPTIONS.SHOW_LABELS
    rois.append(roi)
    
roiwrite('LPS_MALDI_Cortex_alignedROI.zip', rois, mode='w')

In [ ]:
len(rois)

In [ ]:
adata

In [ ]:
adata.write_h5ad("LPS_Scaled_Reg2.h5ad")

In [ ]:
from spatialdata import read_zarr
import geopandas as gpd
import numpy as np
import scanpy as sc
from shapely import Polygon
from read_roi import read_roi_file, read_roi_zip
import pandas as pd

In [ ]:
sdata = read_zarr("/media/duan/sda2/MALDI/Data/20250727_LPS_MALDI/Result/raw_sdata.zarr")
sdata

In [ ]:
st_polygon = sdata.shapes['MultiModel'].copy()
st_polygon

In [ ]:
cortex_roi = read_roi_zip("/media/duan/sda2/MALDI/Data/20250727_LPS_MALDI/LPS_MALDI_Cortex_alignedROI_Final.zip")

In [ ]:
index_list = []
polygon_list = []

for key, value in cortex_roi.items():
    index_list.append(key)
    
    polygon = Polygon(

        np.column_stack([value['x'][0:4], value['y'][0:4]])

    )
    
    polygon_list.append(polygon)

In [ ]:
sm_polygon = gpd.GeoDataFrame(geometry=polygon_list)
sm_polygon

In [ ]:
sm_polygon.index = index_list
sm_polygon

In [ ]:
adata_sm = sc.read_h5ad("/media/duan/sda2/MALDI/Data/LPS_01/LPS_Scaled_Reg2.h5ad")
adata_sm

In [ ]:
adata_sm = adata_sm[sm_polygon.index, :].copy()

In [ ]:
sc.pl.violin(adata_sm, keys = ['total_intensity'])

In [ ]:
sc.pl.violin(adata_sm, keys = ['mean_intensity'])

In [ ]:
np.percentile(adata_sm.obs['mean_intensity'], 0.1)

In [ ]:
np.percentile(adata_sm.obs['mean_intensity'], 99.9)

In [ ]:
np.percentile(adata_sm.obs['total_intensity'], 0.1)

In [ ]:
np.percentile(adata_sm.obs['total_intensity'], 99.9)

In [ ]:
adata_sm = adata_sm[adata_sm.obs['mean_intensity'] > 5].copy()
adata_sm = adata_sm[adata_sm.obs['mean_intensity'] <= 57].copy()
adata_sm = adata_sm[adata_sm.obs['total_intensity'] <= 78690].copy()
adata_sm = adata_sm[adata_sm.obs['total_intensity'] > 7344].copy()
adata_sm

In [ ]:
sm_polygon = sm_polygon.loc[adata_sm.obs_names, :].copy()
sm_polygon

In [ ]:
adata_st = sdata.tables['table'].copy()
adata_st

In [ ]:
areas = []
for pg in sdata.shapes["MultiModel"]["geometry"]:
    areas.append(pg.area)

In [ ]:
adata_st.obs['area'] = areas
adata_st

In [ ]:
NP_Pad = pd.read_csv("/media/duan/sda2/MALDI/Data/20250727_Saline_MALDI/NP_Fold_V58.csv", index_col=0)

In [ ]:
adata_st.var_names = [x[0:-2] if x.endswith("_s") else x for x in adata_st.var_names]

In [ ]:
tmpmatrix = adata_st.to_df().copy()

In [ ]:
tmpmatrix = tmpmatrix.astype('float32')

In [ ]:
import numpy as np

for col in tmpmatrix.columns:
    # tmpmatrix.loc[:,col] = tmpmatrix.loc[:,col] * np.log2(NP_Pad.loc[col, 'Fold'] + 1)
    tmpmatrix.loc[:,col] = tmpmatrix.loc[:,col] * NP_Pad.loc[col, 'Fold']

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
adata_st.layers['counts'] = adata_st.X.copy()
adata_st.X = csr_matrix(tmpmatrix)
adata_st.layers['counts_corrected'] = adata_st.X.copy()

In [ ]:
adata_st

In [ ]:
sc.pp.calculate_qc_metrics(
    adata_st, inplace=True, log1p=True, percent_top=None
)

In [ ]:
sc.pl.violin(
    adata_st,
    ["n_genes_by_counts", "total_counts"],
    jitter=0.4,
    multi_panel=True,
)

In [ ]:
adata_st.obs['total_counts'].mean()

In [ ]:
adata_st.obs['total_counts'].median()

In [ ]:
# Filter cells according to Cell Volume
adata_st.obs["area"].mean()

In [ ]:
np.percentile(adata_st.obs["area"], 1)

In [ ]:
np.percentile(adata_st.obs["area"], 90)

In [ ]:
np.percentile(adata_st.obs["area"], 50)

In [ ]:
adata_st = adata_st[(adata_st.obs["area"] > 500) & (adata_st.obs["area"] < (3 * adata_st.obs["area"].mean())),:].copy()
adata_st

In [ ]:
adata_st.obs['total_counts'].median()

In [ ]:
sc.pp.filter_cells(adata_st, min_genes=2)
sc.pp.filter_cells(adata_st, max_counts=600)
sc.pp.filter_cells(adata_st, min_counts=10)
adata_st

In [ ]:
adata_st.write_h5ad("Result/st_filtered.h5ad")

In [ ]:
st_polygon.index = [str(x) for x in st_polygon.index]

In [ ]:
st_polygon = st_polygon.loc[adata_st.obs_names, :].copy()
st_polygon

In [ ]:
intersection = gpd.sjoin(st_polygon, sm_polygon, how='inner', predicate='intersects')
intersection

## 处理intersection, 并记录相交面积，依据SpaceM

* 细胞分割与MALDI的格子相交面积需占MALDI格子面积的45%以上，约为720个平方像素。这个相交部分称之为 sampling_area  
* 若ablation marker和多个细胞相交，若其中一个cell的sampling_area占总相交面积的90%以上，则保留该ablation marker  
* 所有保留的ablation marker的位置信息改成对应细胞的中心；
* 最终保留下来的ablation marker可以进行一步merge，若两个保留的ablation marker的sampling area来自同一个细胞，直接相加，sampling specificity根据Sampling area加权平均
* 最后根据sampling area进行intensity的矫正，RMS矫正过的信号先根据sampling_area的占ablation marker的比例进行normalize；  

### intersection的index是st的细胞，是排好序的，index_right是sm的ablation marker index

In [ ]:
intersection['index_st'] = intersection.index
intersection['index_sm'] = intersection['index_right'].copy()
intersection

In [ ]:
intersection = intersection.set_index('index_right', drop = True)
intersection

In [ ]:
# 查看ablation marker和细胞的交集情况，先第一遍遍历记录cell_area，ablation_area面积和 sampling area
from tqdm import tqdm

cell_areas = []
ablation_areas = []
sampling_areas = []

for _, row in tqdm(intersection.iterrows()):
    
    st_idx = row['index_st']
    sm_idx = row['index_sm']
    
    ablation_marker = sm_polygon.loc[sm_idx, 'geometry']
    cell_segmentation = st_polygon.loc[st_idx, 'geometry']
    
    cell_areas.append(cell_segmentation.area)
    ablation_areas.append(ablation_marker.area)
    sampling_areas.append(ablation_marker.intersection(cell_segmentation).area)

In [ ]:
intersection['cell_area'] = cell_areas
intersection['ablation_area'] = ablation_areas
intersection['sampling_area'] = sampling_areas

In [ ]:
intersection

In [ ]:
criteria1 = (intersection['sampling_area'] - intersection['ablation_area'] * 0.45) > 0

In [ ]:
intersection['criteria1'] = criteria1

In [ ]:
intersection = intersection.sort_index()
intersection

In [ ]:
# 第二遍遍历记录sampling_specificity

sampling_specificities = []

for idx in tqdm(intersection.index.unique()):
    
    if isinstance(intersection.loc[idx,:], pd.core.frame.Series):
        sampling_specificities.append(1.0)
    else:
        
        tmp_gdf = intersection.loc[idx, :].copy()
        
        batch_total_sampling_area = tmp_gdf['sampling_area'].sum()
        for batch_sampling_area in tmp_gdf['sampling_area']:
            sampling_specificities.append(batch_sampling_area/batch_total_sampling_area)

In [ ]:
intersection['sampling_specificity'] = sampling_specificities
intersection

In [ ]:
criteria2 = intersection['sampling_specificity'] > 0.9
intersection['criteria2'] = criteria2
intersection

In [ ]:
intersection['criteria1'] & intersection['criteria2']

In [ ]:
intersection = intersection[intersection['criteria1'] & intersection['criteria2']].copy()

In [ ]:
intersection

In [ ]:
if len(intersection['index_st'].unique()) < len(intersection):
    print('需要进行Ablation marker merge')

In [ ]:
intersection = intersection.set_index('index_st', drop = False)
intersection

In [ ]:
intersection = intersection.sort_index()
intersection

In [ ]:
# 第三遍遍历merge用了同一个细胞的ablation marker

new_ablation_areas = []
new_sampling_areas = []
new_sampleing_specificities = []

sampling_specificities = []

for idx in tqdm(intersection.index.unique()):
    
    if isinstance(intersection.loc[idx,:], pd.core.frame.Series):
        new_ablation_areas.append(intersection.loc[idx,'ablation_area'])
        new_sampling_areas.append(intersection.loc[idx,'sampling_area'])
        new_sampleing_specificities.append(intersection.loc[idx,'sampling_specificity'])
        
    else:
        
        tmp_gdf = intersection.loc[idx, :].copy()
        
        tmp_ablation_area = tmp_gdf['ablation_area'].sum()
        tmp_sampling_area = tmp_gdf['sampling_area'].sum()
        tmp_sampling_specificity = (tmp_gdf['sampling_area'] * tmp_gdf['sampling_specificity']).sum()/tmp_sampling_area
        
        new_ablation_areas.extend([tmp_ablation_area] * len(tmp_gdf))
        new_sampling_areas.extend([tmp_sampling_area] * len(tmp_gdf))
        new_sampleing_specificities.extend([tmp_sampling_specificity] * len(tmp_gdf))

In [ ]:
intersection['new_ablation_area'] = new_ablation_areas
intersection['new_sampling_area'] = new_sampling_areas
intersection['new_sampling_specificity'] = new_sampleing_specificities

In [ ]:
intersection

In [ ]:
intersection = intersection[['index_st', 'index_sm', 'new_ablation_area', 'new_sampling_area', 'new_sampling_specificity']].copy()

In [ ]:
intersection.columns = ['index_st', 'index_sm', 'ablation_area', 'sampling_area', 'sampling_specificity']

In [ ]:
Final_join_df = intersection.reset_index(drop=True)

In [ ]:
Final_join_df.to_csv("Result/Cortex_join_df.csv", index=False)

In [ ]:
import scanpy as sc
from celllens.utils import *
import os
from tqdm import tqdm
from celllens.preprocessing import *
from celllens.datasets import *
from celllens.celllens import *
import pandas as pd

In [ ]:
adata = sc.read_h5ad("Result/FISH_Cortex_all.h5ad")
adata

In [ ]:
df = adata.to_df(layer = 'counts_corrected')
df

In [ ]:
df['x'] = adata.obs['x']
df['y'] = adata.obs['y']
#df['area'] = adata.obs['area']
df['leiden'] = adata.obs['leiden']

In [ ]:
df

In [ ]:
Rep2_dataset = LENS_Dataset(df,
                              features_list=adata.var_names.to_list(),
                              nbhd_composition=15,
                              feature_neighbor=15,
                              spatial_neighbor=15,
                              path2img=None,
                             use_transform = False) # Not used -  processed image saving location - in later steps

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Rep2_dataset.initialize(cent_x="x", # x loc of cells
                          cent_y="y", # y loc of cells
                          celltype="feature_labels", # default option - run leiden to initialize the clusters
                          pca_components=25, # PCA components to use, user decide 
                          cluster_res=2.0) # leiden initialization resolution

In [ ]:
# check the training label
Rep2_dataset.dual_labels.shape # should be cluster number x 2

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
Rep2_celllens = CellLENS(Rep2_dataset,
                           device,
                           cnn_model=False, ### NOTE here set to no cnn model
                           cnn_latent_dim=128,
                           gnn_latent_dim=32,
                          ) # generally these parameters no need to change

In [ ]:
Rep2_celllens.get_lens_embedding(round=5,
                                   k=32,
                                   learning_rate=1e-3,
                                   n_epochs=5000,
                                   loss_fn='MSELoss',
                                   OptimizerAlg='Adam',
                                   optimizer_kwargs={},
                                   SchedulerAlg=None,
                                   scheduler_kwargs={},
                                   verbose=True) # generally these parameters do not need to change

In [ ]:
# the resulting CellLENS embedding is stored here:
pd.DataFrame(Rep2_celllens.lens_embedding)

In [ ]:
Rep2_celllens.get_lens_clustering(neighbor=15, # standard leiden parameter
                                    resolution=1.0, # resolution for leiden - specific
                                    entropy_threshold=0.75, # CellLENS parameter - generally no change needed
                                    concen_threshold=1, # CellLENS parameter - generally no change needed
                                    max_breaks=3, # CellLENS parameter - generally no change needed
                                    size_lim=50 # CellLENS parameter - smallest cluster size allowed
                                   )

In [ ]:
Rep2_celllens.visualize_umap(Rep2_celllens.lens_embedding, Rep2_celllens.lens_clustering)

In [ ]:
adata.obsm['CellLENS'] = Rep2_celllens.lens_embedding
adata.obs['lens_cluster'] = Rep2_celllens.lens_clustering

In [ ]:
sc.set_figure_params(figsize=(11,6))
sc.pl.embedding(adata, basis = 'spatial', color = 'lens_cluster', size = 3)

In [ ]:
adata.write_h5ad("Result/FISH_Cortex_all_cellLens.h5ad")

In [ ]:
import scanpy as sc
import cosg
import pandas as pd

In [ ]:
adata = sc.read_h5ad("Result/FISH_Cortex_all_cellLens.h5ad")

In [ ]:
adata

In [ ]:
groupby='lens_cluster'
cosg.cosg(adata, key_added='cosg_leiden',
          # use_raw=False, layer='log1p', ## e.g., if you want to use the log1p layer in adata
          mu=1,
          expressed_pct=0.1,
          remove_lowly_expressed=True,
          n_genes_user=10,
          groupby=groupby)

import pandas as pd
pd.DataFrame(adata.uns["cosg_leiden"]["names"]).to_csv('LPS_Cortex_cosg.csv')

In [ ]:
sc.pp.neighbors(adata, use_rep='CellLENS')
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color = ['lens_cluster', 'Penk'], size = 10, wspace = 0.5, groups = ['6'])

In [ ]:
sc.pl.embedding(adata, basis="spatial", color="lens_cluster", size = 10, groups=['7'])

In [ ]:
annotation_dict = {
    '0': 'Microglia',
    '1': 'Ext L6 CT',
    '2': 'Inh Other',
    '3': 'Endo',
    '4': 'Ext L4/5 IT',
    '5': 'Ext L4/5 IT',
    '6': 'Unknown',
    '7': 'Ext L2/3 IT',
    '8': 'Inh Other',
    '9': 'Ext L5 IT',
    '10': 'Mural',
    '11': 'Ext L4 RSP-ACA',
    '12': 'Ext L5 ET/NP',
    '13': 'Astro',
    '14': 'Ext L2/3 IT',
    '15': 'Ext L4/5 IT',
    '16': 'Ext L5 ET/NP',
    '17': 'OPC',
    '18': 'Oligo',
    '19': 'Fibro',
    '20': 'Endo',
    '21': 'Ext L5 IT',
    '22': 'Inh Lamp5',
    '23': 'Inh Pvalb',
    '24': 'Inh Sst',
    '25': 'Ext L4 RSP-ACA',
    '26': 'Inh Vip',
    '27': 'Ext L5 ET/NP',
    '28': 'Ext L5 IT',
    '29': 'Astro',
    '30': 'Fibro',
    '31': 'Endo',
    '32': 'Ext L4/5 IT',
    '33': 'Ext L5 IT',
    '34': 'Unknown',
    '35': 'Oligo',
    '36': 'Unknown',
    '37': 'Inh Sst',
    '38': 'Inh Sst',
    '39': 'Astro',
    '40': 'Astro',
    '41': 'Ext L5 IT',
    '42': 'Ext L4 RSP-ACA',
    '43': 'Ext L4/5 IT',
    '44': 'Ext L5 ET/NP',
    '45': 'Mural',
    '46': 'Ext L4/5 IT',
    '47': 'Endo',
    '48': 'Fibro',
    '49': 'Unknown',
}

In [ ]:
adata.obs['cell_type'] = [annotation_dict[x] for x in adata.obs['lens_cluster']]

In [ ]:
sc.pl.embedding(adata, basis="spatial", color="lens_cluster", size = 10, groups = ['4', '5'])

In [ ]:
import numpy as np

In [ ]:
cell_type_list = np.array([annotation_dict[x] for x in adata.obs['lens_cluster']])

In [ ]:
sc.pl.umap(mdata, color = ['cell_type2'], size = 10)

In [ ]:
sc.pl.umap(adata, color = ['cell_type2'], size = 10)

In [ ]:
adata.obs['cell_type'] = cell_type_list

In [ ]:
sc.pl.embedding(adata, basis="spatial", color="cell_type", size = 10, groups='Macrophage')

In [ ]:
adata.write_h5ad("Result/FISH_Cortex_all_cellLens_annotated.h5ad")

In [ ]:
adata = sc.read_h5ad("Result/FISH_Cortex_all_cellLens_annotated.h5ad")

In [ ]:
adata_sm = sc.read_h5ad("/media/duan/sda2/MALDI/Data/LPS_01/LPS_Scaled_Reg2.h5ad")
adata_sm

In [ ]:
join_df = join_df.set_index('index_st', drop = False)
join_df

In [ ]:
join_df = join_df.loc[[int(x) for x in adata.obs_names], :]

In [ ]:
join_df

In [ ]:
adata_sm = adata_sm[[str(x) for x in join_df['index_sm']]].copy()
adata_sm

In [ ]:
RMS_df = adata_sm.to_df()

In [ ]:
RMS_df

In [ ]:
RMS_df['index_st'] = join_df['index_st'].to_numpy()
RMS_df['index_sm'] = join_df['index_sm'].to_numpy()
# RMS_df['sampling_specificity'] = join_df['sampling_specificity'].to_numpy()
# RMS_df = RMS_df.set_index('index_st', drop = False)
RMS_df

In [ ]:
RMS_df = RMS_df.groupby('index_st').sum()

In [ ]:
RMS_df

In [ ]:
join_df = join_df.drop_duplicates('index_st')

In [ ]:
join_df = join_df.loc[RMS_df.index,:].copy()

In [ ]:
RMS_df = RMS_df * (join_df['sampling_area'] / join_df['ablation_area']).to_numpy().reshape(-1,1)

In [ ]:
RMS_df = RMS_df.iloc[:, 0:-1]

In [ ]:
RMS_df

In [ ]:
adata_sm = sc.AnnData(RMS_df)

In [ ]:
adata.obs

In [ ]:
adata_sm = adata_sm[adata.obs_names,:].copy()
adata_sm.obs['x'] = adata.obs['x']
adata_sm.obs['x'] = adata.obs['x']
adata_sm.obs['cell_type'] = adata.obs['cell_type']
adata_sm.obsm['spatial'] = adata.obsm['spatial']

In [ ]:
adata_sm.write_h5ad("Result/MALDI_Cortex_norm_annotated.h5ad")

In [ ]:
mdata = sc.read_h5ad("Result/MALDI_Cortex_norm_annotated_filtered.h5ad")
mdata

In [ ]:
cidx = [x for x in mdata.obs_names if x in adata.obs_names]

In [ ]:
adata = adata[cidx,:].copy()
mdata = mdata[cidx,:].copy()

In [ ]:
mdata.obs['cell_type2'] = adata.obs['cell_type']

In [ ]:
mdata.obsm['X_umap'] = adata.obsm['X_umap']

In [ ]:
mdata = mdata[mdata.obs['cell_type2'] != 'Unknown'].copy()

In [ ]:
sc.set_figure_params(figsize = (6,6), dpi_save = 300)
sc.pl.umap(mdata, color = ['cell_type2'], frameon = False, save = 'Cortex_FISH_cell_type.pdf', size = 40)

In [ ]:
sc.set_figure_params(figsize = (6,6), dpi_save = 300)
sc.pl.umap(mdata, color = ['772.5256'], frameon = False,size = 40)

In [ ]:
translate_dict = {
    'Astro': 'Astro',
    'Endo': 'Endo',
    'Ext L2/3 IT': 'Ext',
    'Ext L2/3 RSP': 'Ext',
    'Ext L4 RSP-ACA': 'Ext',
    'Ext L4/5 IT': 'Ext',
    'Ext L5 ET/NP': 'Ext',
    'Ext L5 IT': 'Ext',
    'Ext L6 CT': 'Ext',
    'Fibro': 'Fibro',
    'Inh Other': 'Inh',
    'Inh Vip': 'Inh',
    'Inh Lamp5': 'Inh',
    'Inh Pvalb': 'Inh',
    'Inh Sst': 'Inh',
    'Macrophage': 'Macrophage',
    'Microglia': 'Microglia',
    'Mural': 'Mural',
    'OPC': 'OPC',
    'Oligo': 'Oligo'
}

In [ ]:
mdata.obs['cell_class'] = [translate_dict[x] for x in mdata.obs['cell_type2']]

In [ ]:
groupby='cell_class'
cosg.cosg(mdata, key_added='cosg_cell_type',
          # use_raw=False, layer='log1p', ## e.g., if you want to use the log1p layer in adata
          mu=1,
          expressed_pct=0.1,
          remove_lowly_expressed=True,
          n_genes_user=50,
          groupby=groupby)

In [ ]:
import pandas as pd
df = pd.DataFrame(mdata.uns["cosg_cell_type"]["names"])
df

In [ ]:
mdata.obs['cell_class'] = mdata.obs['cell_class'].astype('category')

In [ ]:
sc.tl.dendrogram(mdata, groupby = 'cell_class', use_rep = 'X')

In [ ]:
sc.set_figure_params(scanpy = False, dpi_save=300)
sc.pl.dotplot(mdata, var_names={x: df.loc[0:4, x].to_numpy() for x in df.columns}, groupby='cell_class', 
              standard_scale='var', dendrogram=True, cmap = 'RdYlBu_r' ,figsize=(17, 4), save = 'Cortex_class_MALDI.pdf')

In [ ]:
mdata.write_h5ad("Result/MALDI_Cortex_norm_cellLens_annotated_filtered.h5ad")

In [ ]:
sc.set_figure_params(figsize = (9,6), dpi_save = 300)
sc.pl.embedding(mdata, basis="spatial", color="cell_type2", size = 30, frameon = False, save = 'Cortex_celltype_MALDI.pdf' )

In [ ]:
sc.pp.calculate_qc_metrics(
    mdata, inplace=True, log1p=False, percent_top=None
)

In [ ]:
sc.set_figure_params(figsize = (9,6), dpi_save = 300)
sc.pl.embedding(mdata, basis="spatial", color="total_counts", size = 30, frameon = False, save = 'Cortex_totalIntensity_MALDI.pdf', vmax = 'p99')

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
from shapely import Polygon
from spatialdata import read_zarr,  polygon_query
import geopandas as gpd
from read_roi import read_roi_file, read_roi_zip
import spatialdata_plot
from spatialdata.models import Image2DModel, PointsModel,ShapesModel,TableModel

In [ ]:
sdata = read_zarr("/media/duan/sda2/MALDI/Data/20250727_LPS_MALDI/Result/raw_sdata.zarr")
sdata

In [ ]:
roi = read_roi_file("/media/duan/sda2/MALDI/Data/20250727_LPS_MALDI/Result/Enlarge_Segmentation2.roi")
roi

In [ ]:
roi = roi['Enlarge_Segmentation2']

In [ ]:
polygon_roi = Polygon([[roi['left'], roi['top']], 
                   [roi['left'] + roi['width'], roi['top']], 
                   [roi['left'] + roi['width'], roi['top'] + roi['height']], 
                   [roi['left'], roi['top'] + roi['height']]])
polygon_roi

In [ ]:
cortex_roi = read_roi_zip("/media/duan/sda2/MALDI/Data/20250727_LPS_MALDI/LPS_MALDI_Cortex_alignedROI_Final.zip")
cortex_roi

In [ ]:
index_list = []
polygon_list = []

for key, value in cortex_roi.items():
    index_list.append(key)
    
    polygon = Polygon(

        np.column_stack([value['x'][0:4], value['y'][0:4]])

    )
    
    polygon_list.append(polygon)

In [ ]:
sm_polygon = gpd.GeoDataFrame(geometry=polygon_list)
sm_polygon.index = index_list
sm_polygon

In [ ]:
sdata.shapes['Cortex_ablation'] = ShapesModel.parse(sm_polygon)

In [ ]:
adata_sm = sc.read_h5ad("/media/duan/sda2/MALDI/Data/LPS_01/LPS_Scaled_Reg2.h5ad")
adata_sm

In [ ]:
adata_sm.obs.loc[sm_polygon.index, 'total_intensity']

In [ ]:
total_intensity = adata_sm.obs.loc[sm_polygon.index, 'total_intensity'].to_numpy()

In [ ]:
np.percentile(total_intensity, 99.9)

In [ ]:
total_intensity = np.clip(total_intensity, 0, np.percentile(total_intensity, 99.9))

In [ ]:
sdata.shapes['Cortex_ablation']['total_intensity'] = total_intensity

In [ ]:
from spatialdata import polygon_query
crop1 = lambda x: polygon_query(
    x,
    polygon=polygon_roi,
    target_coordinate_system="global",
)

In [ ]:
adata_st = sc.read_h5ad("/media/duan/sda2/MALDI/Data/20250727_LPS_MALDI/Result/FISH_Cortex_annotated.h5ad")
adata_st

In [ ]:
adata_sm_kept = sc.read_h5ad("/media/duan/sda2/MALDI/Data/20250727_LPS_MALDI/Result/MALDI_Cortex_norm_annotated_filtered.h5ad")
adata_sm_kept.obs

In [ ]:
df = pd.read_csv("/media/duan/sda2/MALDI/Data/20250727_LPS_MALDI/Result/Cortex_join_df.csv")
df

In [ ]:
df = df.set_index('index_st', drop = False)

In [ ]:
df

In [ ]:
df = df.loc[adata_st.obs_names.to_numpy().astype(int),:].copy()

In [ ]:
df['index_sm'].unique().astype(str)

In [ ]:
sdata.shapes['Cortex_ablation'] = sdata.shapes['Cortex_ablation'].loc[df['index_sm'].unique().astype(str), :].copy()

In [ ]:
crop1(sdata).pl.render_shapes("Cortex_ablation", color = 'total_intensity', method = 'matplotlib',
                              outline_alpha = 0, fill_alpha = .9).pl.show(title="Cortex_ablation", figsize = (5,5), dpi = 100, frameon = False, save = 'figures/Ablation_Total_Intensity_enlarge_kept.pdf')

In [ ]:
sdata.shapes['MultiModel'].index = sdata.shapes['MultiModel'].index.astype('str')

In [ ]:
sdata.shapes['MultiModel'] = sdata.shapes['MultiModel'].loc[adata_st.obs_names,:].copy()

In [ ]:
crop1(sdata).pl.render_shapes("Cortex_ablation", color = None, method = 'matplotlib',
                              outline_alpha = 1., fill_alpha = .0, outline_color = '#FFFF00', outline_width = 1.5).pl.render_shapes("MultiModel", color = None, method = 'matplotlib',
                              outline_alpha = 1., fill_alpha = .0, outline_color = '#00BFFF', outline_width = 1.5).pl.show(title="Ablation_Segmentation", figsize = (5,5), dpi = 100, frameon = False, save = 'figures/Ablation_Segmentation_enlarge.pdf')

In [ ]:
sc.pp.calculate_qc_metrics(
    adata_sm_kept, inplace=True, log1p=True, percent_top=None
)

In [ ]:
adata_sm_kept

In [ ]:
total_intensity = adata_sm_kept.obs['total_counts'].to_numpy()

In [ ]:
np.percentile(total_intensity, 99.9)

In [ ]:
total_intensity = np.clip(total_intensity, 0, np.percentile(total_intensity, 99.9))

In [ ]:
sdata.shapes['MultiModel']['total_intensity'] = total_intensity

In [ ]:
sdata.shapes['MultiModel']

In [ ]:
crop1(sdata).pl.render_shapes("MultiModel", color = 'total_intensity', method = 'matplotlib',
                              outline_alpha = 0, fill_alpha = .9).pl.show(title="MultiModel", figsize = (5,5), dpi = 100, frameon = False, save = 'figures/MultiModel_Total_Intensity_enlarge.pdf')

In [ ]:
sdata

In [ ]:
total_intensity = adata_st.obs['total_counts'].to_numpy()

In [ ]:
np.percentile(total_intensity, 99.9)

In [ ]:
total_intensity = np.clip(total_intensity, 0, np.percentile(total_intensity, 99.9))

In [ ]:
sdata.shapes['MultiModel']['total_counts'] = total_intensity

In [ ]:
crop1(sdata).pl.render_shapes("MultiModel", color = 'total_counts', method = 'matplotlib',cmap = 'inferno',
                              outline_alpha = 0, fill_alpha = .9).pl.show(title="MultiModel", figsize = (5,5), dpi = 100, frameon = False, save = 'figures/MultiModel_Total_Counts_enlarge.pdf')